In [ ]:
pip install creversi

In [ ]:
import creversi
from IPython.display import display, SVG

In [ ]:
#ランダムAI

import random

def rm(board, show_step = False):
  move = board.move_from_str(str(random.choice((list([creversi.move_to_str(move) for move in board.legal_moves])))))
  if show_step:
    display(SVG(board.to_svg(move)))
  return board

#ランダムvsランダム
def ar(board, show_step = False):
  print("Random move till end of game")
  while board.is_game_over() == False:
    rm(board, show_step)
  n_white = 64 - board.piece_num()
  n_black = board.piece_num()
  if n_white > n_black:
    print("White Win!")
  elif n_black > n_white:
    print("Black Win!")
  else:
    print("Draw!")
  print("White:",n_white)
  print("Black:",n_black)
  return board

#プレイヤーvsランダムAI
def vsRandom(board):
  while True:
    try:
      user_input = input("Please select who goes first(1: Player, 2:CPU, 3:Random): ")
      turn_order = int(user_input)
      if turn_order == 1:
        player_turn = True
        break
      elif turn_order == 2:
        player_turn = False
        break
      elif turn_order == 3:
        player_turn = random.choice([True, False])
        break
      else:
        print("Invalid input, please try again.")
    except ValueError:
      print("Invalid input, please try again.")

  if player_turn:
    print("Player goes first.")
  else:
    print("CPU goes first.")

  display(board)
  while not board.is_game_over():
    if player_turn:
      board.move(playerInputMove(board))
      player_turn = False
    else:
      print("CPU move: ")
      rm(board, True)
      player_turn = True

  n_white = 64 - board.piece_num()
  n_black = board.piece_num()
  if n_white > n_black:
    print("White Win!")
  elif n_black > n_white:
    print("Black Win!")
  else:
    print("Draw!")
  print("White:",n_white)
  print("Black:",n_black)

  return board

def playerInputMove(board):
  while True:
    try:
      user_input = input("Please input your move: ")
      legal_moves_str = []
      for move in board.legal_moves:
        legal_moves_str.append(creversi.move_to_str(move))
      if user_input in legal_moves_str:
        return creversi.move_from_str(user_input)
      else:
        print("Invalid input, please try again. Legal moves are:")
        for move in board.legal_moves:
          print(creversi.move_to_str(move))
    except ValueError:
      print("Invalid input, please try again.")

In [ ]:
board = creversi.Board()
vsRandom(board)